# HW_5

In [13]:
import pandas as pd
import os
import zipfile

# Розпакування архіву
with zipfile.ZipFile("homework.zip", "r") as zip_ref:
    zip_ref.extractall("homework")

# Функція для завантаження даних з папки та об'єднання їх у DataFrame
def load_data(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            activity = os.path.basename(folder_path)
            df['activity'] = activity
            data.append(df)
    return pd.concat(data, ignore_index=True)

# Функція для розрахунку часових ознак
def calculate_time_features(data):
    time_features = data.groupby('activity').agg(['mean', 'std', 'min', 'max'])
    return time_features

# Шляхи до папок з даними
data_folders = ['homework/data/idle', 'homework/data/running', 'homework/data/stairs', 'homework/data/walking']

# Завантаження та обробка даних
all_data = pd.concat([load_data(folder) for folder in data_folders], ignore_index=True)

# Розрахунок часових ознак
time_features = calculate_time_features(all_data)

print(time_features)

         accelerometer_X                                 accelerometer_Y  \
                    mean       std        min        max            mean   
activity                                                                   
idle            0.096767  0.691892 -10.448297   8.135497        2.225971   
running         5.045705  9.952696 -39.188293  39.188293        8.083135   
stairs          0.353135  3.845949 -11.238382  14.164097       -9.574353   
walking        -2.661943  4.648378 -26.815060  20.575770       -9.703702   

                                          accelerometer_Z            \
                std        min        max            mean       std   
activity                                                              
idle       2.821235  -2.595315   9.916783        8.973247  1.211058   
running   12.441402 -39.188293  39.188293        1.387356  7.004236   
stairs     4.922294 -25.828648   4.994305       -1.841333  5.297376   
walking    5.931165 -39.188293  14.441825

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [15]:
# Розділення даних на ознаки (X) та цільову змінну (y)
X = all_data.drop('activity', axis=1)
y = all_data['activity']
# Розділення даних на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Нормалізувати дані
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# Навчання моделі SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)


SVC(kernel='linear') In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. SVC SVC(kernel='linear')

In [30]:
# Навчання моделі випадкового лісу

# rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # Виникло перенавчання
# зменшуємо кількість дерев до 50 і обмежили глибину кожного дерева до 10
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. RandomForestClassifier RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42)

In [31]:
# Оцінка моделі SVM
svm_predictions = svm_model.predict(X_test_scaled)
svm_report = classification_report(y_test, svm_predictions, zero_division=1)
print("SVM Classification Report:")
print(svm_report)

# Оцінка моделі випадкового лісу
rf_predictions = rf_model.predict(X_test_scaled)
rf_report = classification_report(y_test, rf_predictions, zero_division=1)
print("\nRandom Forest Classification Report:")
print(rf_report)

SVM Classification Report:
              precision    recall  f1-score   support

        idle       0.80      0.77      0.79      6203
     running       0.85      0.85      0.85     20485
      stairs       1.00      0.00      0.00      1022
     walking       0.76      0.86      0.81     11062

    accuracy                           0.82     38772
   macro avg       0.85      0.62      0.61     38772
weighted avg       0.82      0.82      0.81     38772


Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       0.99      0.99      0.99      6203
     running       0.95      0.95      0.95     20485
      stairs       1.00      0.08      0.15      1022
     walking       0.86      0.95      0.90     11062

    accuracy                           0.93     38772
   macro avg       0.95      0.74      0.75     38772
weighted avg       0.93      0.93      0.92     38772



In [32]:
# Модель Random Forest показує вищу точність загалом, проте має проблеми з класифікацією класу "stairs". 
# Модель SVM має проблеми з точністю класифікації класу "stairs", але може бути корисною, якщо точність для цього класу не є критичною.
# Обидві моделі погано класифікують клас "stairs", 
# що може бути пов'язано з недостатнім обсягом даних для цього класу або з важкістю відокремлення класу "stairs" від інших класів